In [1]:
import pandas as pd
import sqlite3 
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline
import warnings
import time
import calendar
from datetime import datetime as dt
from adtk.data import validate_series
warnings.filterwarnings("ignore")

In [2]:
!pip install ipython-sql

Defaulting to user installation because normal site-packages is not writeable


In [3]:
#Joining dataset and reading csv to pandas dataframe:
file_path = ['202209-divvy-publictripdata.csv', '202210-divvy-tripdata.csv', '202211-divvy-tripdata.csv', '202212-divvy-tripdata.csv', 
             '202301-divvy-tripdata.csv', '202302-divvy-tripdata.csv']
dataframes = [pd.read_csv(file) for file in file_path]

merged_df = pd.concat(dataframes, ignore_index = True)
print(merged_df)

                  ride_id  rideable_type           started_at  \
0        5156990AC19CA285  electric_bike  2022-09-01 08:36:22   
1        E12D4A16BF51C274  electric_bike  2022-09-01 17:11:29   
2        A02B53CD7DB72DD7  electric_bike  2022-09-01 17:15:50   
3        C82E05FEE872DF11  electric_bike  2022-09-01 09:00:28   
4        4DEEB4550A266AE1  electric_bike  2022-09-01 07:30:11   
...                   ...            ...                  ...   
2160306  C04510F8EBB5EE8A   classic_bike  2023-02-08 21:57:22   
2160307  187BA364FB265C80  electric_bike  2023-02-19 11:29:09   
2160308  46B54F6B417D1B27  electric_bike  2023-02-07 09:01:33   
2160309  335B3CAD59F6C016  electric_bike  2023-02-22 08:33:22   
2160310  03D59518BB151EFA   classic_bike  2023-02-01 21:52:17   

                    ended_at         start_station_name start_station_id  \
0        2022-09-01 08:39:05                        NaN              NaN   
1        2022-09-01 17:14:45                        NaN            

In [4]:
merged_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5156990AC19CA285,electric_bike,2022-09-01 08:36:22,2022-09-01 08:39:05,NaN,NaN,California Ave & Milwaukee Ave,13084,41.93,-87.69,41.92,-87.70,casual
1,E12D4A16BF51C274,electric_bike,2022-09-01 17:11:29,2022-09-01 17:14:45,NaN,NaN,NaN,NaN,41.87,-87.62,41.87,-87.62,casual
2,A02B53CD7DB72DD7,electric_bike,2022-09-01 17:15:50,2022-09-01 17:16:12,NaN,NaN,NaN,NaN,41.87,-87.62,41.87,-87.62,casual
3,C82E05FEE872DF11,electric_bike,2022-09-01 09:00:28,2022-09-01 09:10:32,NaN,NaN,NaN,NaN,41.93,-87.69,41.94,-87.67,casual
4,4DEEB4550A266AE1,electric_bike,2022-09-01 07:30:11,2022-09-01 07:32:36,NaN,NaN,NaN,NaN,41.92,-87.73,41.92,-87.73,casual


In [5]:
merged_df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [6]:
merged_df ['ride_id'] = merged_df ['ride_id'].astype(str)
merged_df ['rideable_type'] = merged_df ['rideable_type'].astype(str)

In [7]:
merged_df.describe()

,start_lat,start_lng,end_lat,end_lng
count,"2,160,311.00","2,160,311.00","2,158,523.00","2,158,523.00"
mean,41.90,-87.65,41.90,-87.65
std,0.05,0.03,0.09,0.17
min,41.64,-87.84,0.00,-87.92
25%,41.88,-87.66,41.88,-87.66
50%,41.90,-87.64,41.90,-87.65
75%,41.93,-87.63,41.93,-87.63
max,42.07,-87.52,42.15,0.00


In [8]:
merged_df['ride_id'].describe()

count              2160311
unique             2160311
top       5156990AC19CA285
freq                     1
Name: ride_id, dtype: object

In [9]:
rideidlength = merged_df.groupby(merged_df['ride_id'].str.len())['ride_id'].count()
print(rideidlength)

ride_id
16    2160311
Name: ride_id, dtype: int64


In [10]:
merged_df['rideable_type'].unique(), merged_df['rideable_type'].value_counts()

(array(['electric_bike', 'classic_bike', 'docked_bike'], dtype=object),
 rideable_type
 electric_bike    1198292
 classic_bike      917835
 docked_bike        44184
 Name: count, dtype: int64)

In [11]:
merged_df['rideable_type'] = merged_df['rideable_type'].str.replace('_','')
merged_df['rideable_type'].head(2)

0    electricbike
1    electricbike
Name: rideable_type, dtype: object

In [12]:
merged_df[merged_df['rideable_type'] == 'dockedbike'].head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
10882,DFD2D8549F460DB5,dockedbike,2022-09-17 19:46:33,2022-10-01 18:51:47,California Ave & Milwaukee Ave,13084,NaN,NaN,41.92,-87.70,NaN,NaN,casual
11232,BD873237D91827B5,dockedbike,2022-09-04 19:21:23,2022-09-04 20:26:07,Vernon Ave & 107th St,20109,Vernon Ave & 107th St,20109,41.70,-87.61,41.70,-87.61,casual


In [13]:
merged_df.columns = merged_df.columns.str.replace('_','')
merged_df.columns

Index(['rideid', 'rideabletype', 'startedat', 'endedat', 'startstationname',
       'startstationid', 'endstationname', 'endstationid', 'startlat',
       'startlng', 'endlat', 'endlng', 'membercasual'],
      dtype='object')

In [14]:
merged_df['startedat'].isnull().sum() , merged_df['endedat'].isnull().sum()

(0, 0)

In [15]:
merged_df['startedat'].count()

2160311

In [16]:
merged_df ['startedat'] = pd.to_datetime(merged_df['startedat'])

In [17]:
startlength = merged_df['startedat'].apply(lambda x: len(str(x))) 
countstart = startlength.value_counts()
countstart

startedat
19    2160311
Name: count, dtype: int64

In [18]:
ststationcount = merged_df['startstationname'].count()
ststationcount

1831742

In [19]:
merged_df['endedat'].count()

2160311

In [20]:
merged_df ['endedat'] = pd.to_datetime(merged_df['endedat'])

In [21]:
endlength = merged_df['endedat'].apply(lambda x: len(str(x))) 
countend = endlength.value_counts()
countend

endedat
19    2160311
Name: count, dtype: int64

In [22]:
durationalter = (merged_df['startedat'] > merged_df['endedat']).sum()
durationalter

55

In [23]:
merged_df.loc[merged_df['startedat'] > merged_df['endedat'], ['startedat', 'endedat']] = \
    merged_df.loc[merged_df['startedat'] > merged_df['endedat'], ['endedat', 'startedat']].values

In [24]:
swapmerged_df = (merged_df['startedat'] > merged_df['endedat']).sum()
swapmerged_df

0

In [25]:
duration_minutes = ((merged_df['endedat'] - merged_df['startedat']).dt.total_seconds() / 60)
long_durations = (merged_df[duration_minutes >= 1440]).count()
long_durations

rideid              1615
rideabletype        1615
startedat           1615
endedat             1615
startstationname    1615
startstationid      1615
endstationname        30
endstationid          30
startlat            1615
startlng            1615
endlat                50
endlng                50
membercasual        1615
dtype: int64

In [26]:
duration_minutes = ((merged_df['endedat'] - merged_df['startedat']).dt.total_seconds() / 60)
short_durations = (merged_df[duration_minutes <= 1]).count()
short_durations

rideid              53773
rideabletype        53773
startedat           53773
endedat             53773
startstationname    39349
startstationid      39342
endstationname      34172
endstationid        34169
startlat            53773
startlng            53773
endlat              53773
endlng              53773
membercasual        53773
dtype: int64

In [27]:
ststationcount = merged_df['startstationname'].count()
ststationcount

1831742

In [28]:
stationcount = merged_df['startstationname'].value_counts()
station_remove = stationcount.index[stationcount <= 328]
station_remove

Index(['Public Rack - Kostner Ave & Armitage Ave',
       'Milwaukee Ave & Cuyler Ave', 'Chicago State University',
       'Ridge Blvd & Touhy Ave', 'DuSable Museum', 'Whipple/Irving Park',
       'Pulaski Rd & Eddy St (Temp)', 'Kenosha & Wellington',
       'Dorchester Ave & 63rd St', 'Malcolm X College Vaccination Site',
       ...
       'Walden Pkwy & 99th St', 'Avenue L & 114th St',
       'Public Rack - East End Ave & 75th St', 'Ashland Ave & 76th St',
       'William Rainey Harper High School',
       'Public Rack - Oakley Ave & 50th Pl',
       'Public Rack - Richmond St & Thorndale Ave',
       'Public Rack - Union Ave & 111th St', 'Calumet Ave & 103rd St',
       'Public Rack - Halsted St & 80th St'],
      dtype='object', name='startstationname', length=852)

In [29]:
merged_df = merged_df.drop(merged_df[merged_df['startstationname'].isin(station_remove)].index)

In [30]:
enstationcount = merged_df['endstationname'].count()
enstationcount

1784107

In [31]:
enstationcount = merged_df['endstationname'].value_counts()
enstation_remove = enstationcount.index[enstationcount <= 220]
enstation_remove

Index(['Narragansett & Wrightwood', 'Whipple/Irving Park',
       'Public Rack - Kostner Ave & Armitage Ave',
       'Sacramento Blvd & Franklin Blvd', 'W Washington Blvd & N Peoria St',
       'Campbell Ave & Montrose Ave (Temp)', 'Valli Produce - Evanston Plaza',
       'Wolcott Ave & Fargo Ave', 'Lincolnwood Dr & Central St',
       'Fairfield Ave & Roosevelt Rd',
       ...
       'Public Rack - Halsted St & 102nd St',
       'Public Rack - Kedzie Ave & Pershing Rd',
       'Public Rack - Green St & 103rd St',
       'Public Rack - Cicero Ave & Le Moyne St - midblock',
       'Public Rack - Cottage Grove Ave & 111th St', 'Avenue O & 118th St',
       'Vernon Ave & 79th St', 'Public Rack - 10557 S Western Ave',
       'Langley Ave & 79th St', 'Public Rack - James Madison School'],
      dtype='object', name='endstationname', length=767)

In [32]:
merged_df = merged_df.drop(merged_df[merged_df['endstationname'].isin(enstation_remove)].index)

In [33]:
enstationcount = merged_df['endstationname'].count()
enstationcount

1762907

In [34]:
merged_df.isnull().sum()

rideid                   0
rideabletype             0
startedat                0
endedat                  0
startstationname    318657
startstationid      318657
endstationname      334546
endstationid        334546
startlat                 0
startlng                 0
endlat                1499
endlng                1499
membercasual             0
dtype: int64

In [35]:
slat = merged_df['startlat'].isnull().sum()
slng = merged_df['startlng'].isnull().sum()
elat = merged_df['endlat'].isnull().sum()
elng = merged_df['endlng'].isnull().sum()
print ("Start lat null count:", slat, "Start lng null count:", slng, "End lat null count:", elat, "End lng null count:", elng, ".")

Start lat null count: 0 Start lng null count: 0 End lat null count: 1499 End lng null count: 1499 .


In [36]:
merged_df['startstationid'].isnull().sum(), merged_df['startstationname'].isnull().sum()

(318657, 318657)

In [37]:
merged_df['startstationname'].count(), merged_df['startstationid'].count()

(1778796, 1778796)

In [38]:
filtered_df = merged_df[merged_df['startstationid'] == 'KA1504000106'][['startstationid', 'startstationname']]
filtered_df

,startstationid,startstationname
22,KA1504000106,Kedzie Ave & Lake St
1647,KA1504000106,Kedzie Ave & Lake St
1989,KA1504000106,Kedzie Ave & Lake St
6393,KA1504000106,Kedzie Ave & Lake St
6630,KA1504000106,Kedzie Ave & Lake St
...,...,...
2157117,KA1504000106,Kedzie Ave & Lake St
2157430,KA1504000106,Kedzie Ave & Lake St
2157455,KA1504000106,Kedzie Ave & Lake St
2157456,KA1504000106,Kedzie Ave & Lake St


In [39]:
startstation_df = merged_df[merged_df['startstationname'].notnull() & merged_df['startstationid'].notnull()][['startstationname', 'startstationid']]
startstation_df 

,startstationname,startstationid
22,Kedzie Ave & Lake St,KA1504000106
77,California Ave & Division St,13256
80,Western Ave & Roscoe St,15634
231,Western Ave & Roscoe St,15634
376,Morgan Ave & 14th Pl,TA1306000002
...,...,...
2160306,Clark St & Wrightwood Ave,TA1305000014
2160307,Ogden Ave & Roosevelt Rd,KA1504000101
2160308,Clark St & Wrightwood Ave,TA1305000014
2160309,Clark St & Wrightwood Ave,TA1305000014


In [40]:
merged_df['startstationname'].describe()

count                     1778796
unique                        518
top       Streeter Dr & Grand Ave
freq                        19254
Name: startstationname, dtype: object

In [41]:
endstation_df = merged_df[merged_df['endstationname'].notnull() & merged_df['endstationid'].notnull()][['endstationname','endstationid']]
endstation_df

,endstationname,endstationid
0,California Ave & Milwaukee Ave,13084
14,California Ave & Milwaukee Ave,13084
45,Wood St & Chicago Ave,637
150,California Ave & Division St,13256
228,California Ave & Milwaukee Ave,13084
...,...,...
2160306,Sheffield Ave & Waveland Ave,TA1307000126
2160307,Delano Ct & Roosevelt Rd,KA1706005007
2160308,Canal St & Madison St,13341
2160309,Canal St & Madison St,13341


In [42]:
merged_df['endstationname'].describe()

count                     1762907
unique                        522
top       Streeter Dr & Grand Ave
freq                        19396
Name: endstationname, dtype: object

In [43]:
merged_df['startstationname'].value_counts()

startstationname
Streeter Dr & Grand Ave      19254
Ellis Ave & 60th St          14341
University Ave & 57th St     14226
Clark St & Elm St            13631
Kingsbury St & Kinzie St     13458
                             ...  
Long & Irving Park             303
Pulaski Rd & 51st St           213
Narragansett & Wrightwood      198
Princeton Ave & 47th St        141
State St & 95th St              87
Name: count, Length: 518, dtype: int64

In [44]:
startcount = merged_df['startstationname'].value_counts()
filteredstcount = startcount[startcount.index.str.contains('Public Rack')]
filteredstcount

Series([], Name: count, dtype: int64)

In [45]:
endcount = merged_df['endstationname'].value_counts()
filteredendcount = endcount[endcount.index.str.contains('Public Rack')]
filteredendcount

Series([], Name: count, dtype: int64)

In [46]:
filtered_df = merged_df[~merged_df['endstationname'].isin(filteredendcount.index)]
filteredendcount

Series([], Name: count, dtype: int64)

In [47]:
startstation_null = merged_df[merged_df['startstationname'].notnull()]['startstationname']
startstation_null.to_frame()

,startstationname
22,Kedzie Ave & Lake St
77,California Ave & Division St
80,Western Ave & Roscoe St
231,Western Ave & Roscoe St
376,Morgan Ave & 14th Pl
...,...
2160306,Clark St & Wrightwood Ave
2160307,Ogden Ave & Roosevelt Rd
2160308,Clark St & Wrightwood Ave
2160309,Clark St & Wrightwood Ave


In [48]:
startendstation = merged_df[merged_df[['startstationname', 'endstationname']].notnull().all(axis=1)][['startstationname', 'endstationname']]
startendstation

,startstationname,endstationname
387,California Ave & Milwaukee Ave,California Ave & Milwaukee Ave
388,California Ave & Milwaukee Ave,California Ave & Milwaukee Ave
400,Wood St & Chicago Ave,Wood St & Chicago Ave
964,California Ave & Milwaukee Ave,California Ave & Milwaukee Ave
971,California Ave & Division St,California Ave & Division St
...,...,...
2160306,Clark St & Wrightwood Ave,Sheffield Ave & Waveland Ave
2160307,Ogden Ave & Roosevelt Rd,Delano Ct & Roosevelt Rd
2160308,Clark St & Wrightwood Ave,Canal St & Madison St
2160309,Clark St & Wrightwood Ave,Canal St & Madison St


In [49]:
ssn = merged_df['startstationname'].isnull().sum()
esn = merged_df['endstationname'].isnull().sum()
ssi = merged_df['startstationid'].isnull().sum()
esi = merged_df['endstationid'].isnull().sum()
print ("Start station name null count:", ssn, "End station null count:", esn, "Start station id null count:", ssi, "End station id null count:", esi, ".")

Start station name null count: 318657 End station null count: 334546 Start station id null count: 318657 End station id null count: 334546 .


In [50]:
filtered_df = merged_df[merged_df['startstationname'].notnull() & 
merged_df['endstationname'].notnull() & 
merged_df['startstationid'].notnull() & 
merged_df['endstationid'].notnull() ]
filtered_df

,rideid,rideabletype,startedat,endedat,startstationname,startstationid,endstationname,endstationid,startlat,startlng,endlat,endlng,membercasual
387,8C94D4680E7B1DBC,electricbike,2022-09-10 22:52:54,2022-09-10 22:53:43,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.92,-87.70,41.92,-87.70,casual
388,7FD174E61EBA0B99,electricbike,2022-09-10 22:58:27,2022-09-10 22:59:20,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.92,-87.70,41.92,-87.70,casual
400,B23F6CCA8AC89F52,electricbike,2022-09-11 18:35:31,2022-09-11 18:45:57,Wood St & Chicago Ave,637,Wood St & Chicago Ave,637,41.90,-87.67,41.90,-87.67,casual
964,2A39814848E1D54C,electricbike,2022-09-07 20:01:55,2022-09-07 20:19:24,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.92,-87.70,41.92,-87.70,casual
971,181A283DC7877573,electricbike,2022-09-08 17:00:30,2022-09-08 17:02:01,California Ave & Division St,13256,California Ave & Division St,13256,41.90,-87.70,41.90,-87.70,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160306,C04510F8EBB5EE8A,classicbike,2023-02-08 21:57:22,2023-02-08 22:08:06,Clark St & Wrightwood Ave,TA1305000014,Sheffield Ave & Waveland Ave,TA1307000126,41.93,-87.64,41.95,-87.65,member
2160307,187BA364FB265C80,electricbike,2023-02-19 11:29:09,2023-02-19 11:39:11,Ogden Ave & Roosevelt Rd,KA1504000101,Delano Ct & Roosevelt Rd,KA1706005007,41.87,-87.68,41.87,-87.63,member
2160308,46B54F6B417D1B27,electricbike,2023-02-07 09:01:33,2023-02-07 09:16:53,Clark St & Wrightwood Ave,TA1305000014,Canal St & Madison St,13341,41.93,-87.64,41.88,-87.64,casual
2160309,335B3CAD59F6C016,electricbike,2023-02-22 08:33:22,2023-02-22 08:50:11,Clark St & Wrightwood Ave,TA1305000014,Canal St & Madison St,13341,41.93,-87.64,41.88,-87.64,casual


In [51]:
filtered_df.describe()

,startedat,endedat,startlat,startlng,endlat,endlng
count,1599006,1599006,"1,599,006.00","1,599,006.00","1,599,006.00","1,599,006.00"
mean,2022-11-03 20:00:03.675006976,2022-11-03 20:14:05.448695808,41.90,-87.64,41.90,-87.64
min,2022-09-01 00:00:05,2022-09-01 00:02:47,41.77,-87.79,0.00,-87.76
25%,2022-09-23 10:27:41.249999872,2022-09-23 10:40:54.750000128,41.88,-87.66,41.88,-87.66
50%,2022-10-22 10:44:33,2022-10-22 11:02:01,41.90,-87.64,41.90,-87.64
75%,2022-12-05 06:51:16,2022-12-05 06:58:49,41.93,-87.63,41.93,-87.63
max,2023-02-28 23:59:31,2023-03-01 08:58:25,42.06,-87.57,42.06,0.00
std,NaN,NaN,0.04,0.02,0.10,0.20


In [52]:
slat = filtered_df['startlat'].isnull().sum()
slng = filtered_df['startlng'].isnull().sum()
elat = filtered_df['endlat'].isnull().sum()
elng = filtered_df['endlng'].isnull().sum()
memcas = filtered_df['membercasual'].isnull().sum()
print ("Start lat null count:", slat, "Start lng null count:", slng, "End lat null count:", elat, "End lng null count:", elng, ". Also membercasual :", memcas, "null values.")

Start lat null count: 0 Start lng null count: 0 End lat null count: 0 End lng null count: 0 . Also membercasual : 0 null values.


In [53]:
startcount = merged_df['startstationname'].value_counts()  # Assuming merged_df is now defined
filteredstcount = startcount[startcount <= 330 ]
filteredstcount

startstationname
Elston Ave & Henderson St    326
Kedzie Ave & 48th Pl         326
California Ave & Lake St     317
Long & Irving Park           303
Pulaski Rd & 51st St         213
Narragansett & Wrightwood    198
Princeton Ave & 47th St      141
State St & 95th St            87
Name: count, dtype: int64

In [54]:
filtered_df.isnull().sum()

rideid              0
rideabletype        0
startedat           0
endedat             0
startstationname    0
startstationid      0
endstationname      0
endstationid        0
startlat            0
startlng            0
endlat              0
endlng              0
membercasual        0
dtype: int64

In [55]:
filtered_df['month'] = filtered_df['startedat'].dt.month
filtered_df['dayofweek'] = filtered_df['startedat'].dt.dayofweek
filtered_df['starthour'] = filtered_df['startedat'].dt.hour
filtered_df['endhour'] = filtered_df['endedat'].dt.hour

In [56]:
filtered_df['durationmins'] = (filtered_df ['endedat'] - filtered_df ['startedat']).dt.total_seconds()/60
filtered_df.head()

,rideid,rideabletype,startedat,endedat,startstationname,startstationid,endstationname,endstationid,startlat,startlng,endlat,endlng,membercasual,month,dayofweek,starthour,endhour,durationmins
387,8C94D4680E7B1DBC,electricbike,2022-09-10 22:52:54,2022-09-10 22:53:43,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.92,-87.70,41.92,-87.70,casual,9,5,22,22,0.82
388,7FD174E61EBA0B99,electricbike,2022-09-10 22:58:27,2022-09-10 22:59:20,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.92,-87.70,41.92,-87.70,casual,9,5,22,22,0.88
400,B23F6CCA8AC89F52,electricbike,2022-09-11 18:35:31,2022-09-11 18:45:57,Wood St & Chicago Ave,637,Wood St & Chicago Ave,637,41.90,-87.67,41.90,-87.67,casual,9,6,18,18,10.43
964,2A39814848E1D54C,electricbike,2022-09-07 20:01:55,2022-09-07 20:19:24,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.92,-87.70,41.92,-87.70,casual,9,2,20,20,17.48
971,181A283DC7877573,electricbike,2022-09-08 17:00:30,2022-09-08 17:02:01,California Ave & Division St,13256,California Ave & Division St,13256,41.90,-87.70,41.90,-87.70,casual,9,3,17,17,1.52


In [57]:
filtered_df = filtered_df[(filtered_df['durationmins'] >= 1) & (filtered_df['durationmins'] <= 1440)]
filtered_df

,rideid,rideabletype,startedat,endedat,startstationname,startstationid,endstationname,endstationid,startlat,startlng,endlat,endlng,membercasual,month,dayofweek,starthour,endhour,durationmins
400,B23F6CCA8AC89F52,electricbike,2022-09-11 18:35:31,2022-09-11 18:45:57,Wood St & Chicago Ave,637,Wood St & Chicago Ave,637,41.90,-87.67,41.90,-87.67,casual,9,6,18,18,10.43
964,2A39814848E1D54C,electricbike,2022-09-07 20:01:55,2022-09-07 20:19:24,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.92,-87.70,41.92,-87.70,casual,9,2,20,20,17.48
971,181A283DC7877573,electricbike,2022-09-08 17:00:30,2022-09-08 17:02:01,California Ave & Division St,13256,California Ave & Division St,13256,41.90,-87.70,41.90,-87.70,casual,9,3,17,17,1.52
1009,ACA9081DE3E5030A,classicbike,2022-09-10 14:14:09,2022-09-10 14:26:57,California Ave & Division St,13256,California Ave & Division St,13256,41.90,-87.70,41.90,-87.70,casual,9,5,14,14,12.80
1638,192B8C5D116C3D76,classicbike,2022-09-03 19:58:59,2022-09-03 21:09:18,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.92,-87.70,41.92,-87.70,casual,9,5,19,21,70.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160306,C04510F8EBB5EE8A,classicbike,2023-02-08 21:57:22,2023-02-08 22:08:06,Clark St & Wrightwood Ave,TA1305000014,Sheffield Ave & Waveland Ave,TA1307000126,41.93,-87.64,41.95,-87.65,member,2,2,21,22,10.73
2160307,187BA364FB265C80,electricbike,2023-02-19 11:29:09,2023-02-19 11:39:11,Ogden Ave & Roosevelt Rd,KA1504000101,Delano Ct & Roosevelt Rd,KA1706005007,41.87,-87.68,41.87,-87.63,member,2,6,11,11,10.03
2160308,46B54F6B417D1B27,electricbike,2023-02-07 09:01:33,2023-02-07 09:16:53,Clark St & Wrightwood Ave,TA1305000014,Canal St & Madison St,13341,41.93,-87.64,41.88,-87.64,casual,2,1,9,9,15.33
2160309,335B3CAD59F6C016,electricbike,2023-02-22 08:33:22,2023-02-22 08:50:11,Clark St & Wrightwood Ave,TA1305000014,Canal St & Madison St,13341,41.93,-87.64,41.88,-87.64,casual,2,2,8,8,16.82


In [58]:
#0 is Monday, 6 is Sunday
dayofweek = filtered_df['dayofweek'].unique(), filtered_df['dayofweek'].value_counts()
dayofweek

(array([6, 2, 3, 5, 4, 1, 0], dtype=int32),
 dayofweek
 3    243920
 5    236145
 2    228137
 4    227652
 1    223980
 0    214005
 6    194432
 Name: count, dtype: int64)

In [59]:
cnn = sqlite3.connect('simsimdb.db')

In [60]:
filtered_df.to_sql('cyclisticb', cnn, if_exists='replace')

1568271

In [61]:
%load_ext sql

In [62]:
%sql sqlite:///simsimdb.db

In [63]:
%%sql
SELECT *
FROM 
cyclisticb
limit 10;

 * sqlite:///simsimdb.db
Done.


index,rideid,rideabletype,startedat,endedat,startstationname,startstationid,endstationname,endstationid,startlat,startlng,endlat,endlng,membercasual,month,dayofweek,starthour,endhour,durationmins
400,B23F6CCA8AC89F52,electricbike,2022-09-11T18:35:31,2022-09-11T18:45:57,Wood St & Chicago Ave,637,Wood St & Chicago Ave,637,41.89561,-87.672195,41.895634,-87.672069,casual,9,6,18,18,10.433333333333334
964,2A39814848E1D54C,electricbike,2022-09-07T20:01:55,2022-09-07T20:19:24,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.922662333333335,-87.69710816666667,41.922695,-87.697153,casual,9,2,20,20,17.483333333333334
971,181A283DC7877573,electricbike,2022-09-08T17:00:30,2022-09-08T17:02:01,California Ave & Division St,13256,California Ave & Division St,13256,41.903035879,-87.697476506,41.903029,-87.697474,casual,9,3,17,17,1.5166666666666666
1009,ACA9081DE3E5030A,classicbike,2022-09-10T14:14:09,2022-09-10T14:26:57,California Ave & Division St,13256,California Ave & Division St,13256,41.903029,-87.697474,41.903029,-87.697474,casual,9,5,14,14,12.8
1638,192B8C5D116C3D76,classicbike,2022-09-03T19:58:59,2022-09-03T21:09:18,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.922695,-87.697153,41.922695,-87.697153,casual,9,5,19,21,70.31666666666666
1647,D4CD3DF412980C2A,classicbike,2022-09-03T23:18:10,2022-09-03T23:34:21,Kedzie Ave & Lake St,KA1504000106,California Ave & Division St,13256,41.88460328501,-87.7063037436,41.903029,-87.697474,casual,9,5,23,23,16.183333333333334
2201,FB3F59FA0A9FAD8D,classicbike,2022-09-23T21:55:04,2022-09-23T21:56:14,California Ave & Division St,13256,California Ave & Division St,13256,41.903029,-87.697474,41.903029,-87.697474,casual,9,4,21,21,1.1666666666666667
2237,A6FA0AAB756D6F13,classicbike,2022-09-08T18:37:19,2022-09-08T19:41:46,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,casual,9,3,18,19,64.45
2581,0CDA5583BE8E3757,electricbike,2022-09-20T20:09:28,2022-09-20T20:16:09,California Ave & Milwaukee Ave,13084,California Ave & Division St,13256,41.922672,-87.697124,41.903029,-87.697474,casual,9,1,20,20,6.683333333333334
2594,32B5F2F44C16580C,classicbike,2022-09-10T10:26:16,2022-09-10T10:32:25,Wood St & Chicago Ave,637,Wood St & Chicago Ave,637,41.895634,-87.672069,41.895634,-87.672069,casual,9,5,10,10,6.15


In [64]:
%%sql
UPDATE cyclisticb
SET dayofweek = CASE dayofweek
                  WHEN '0' THEN 'Monday'
                  WHEN '1' THEN 'Tuesday'
                  WHEN '2' THEN 'Wednesday'
                  WHEN '3' THEN 'Thursday'
                  WHEN '4' THEN 'Friday'
                  WHEN '5' THEN 'Saturday'
                  WHEN '6' THEN 'Sunday'
                  ELSE dayofweek  
                END
WHERE dayofweek IN ('0', '1', '2', '3', '4', '5', '6');

 * sqlite:///simsimdb.db
1568271 rows affected.


[]

In [65]:
%%sql
UPDATE cyclisticb
SET month = CASE month
                  WHEN '1' THEN 'January'
                  WHEN '2' THEN 'February'
                  WHEN '3' THEN 'March'
                  WHEN '4' THEN 'April'
                  WHEN '5' THEN 'May'
                  WHEN '6' THEN 'June'
                  WHEN '7' THEN 'July'
                  WHEN '8' THEN 'August'
                  WHEN '9' THEN 'September'
                  WHEN '10' THEN 'October'
                  WHEN '11' THEN 'November'
                  WHEN '12' THEN 'December'
                  ELSE month  
                END
WHERE month IN ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12');

 * sqlite:///simsimdb.db
1568271 rows affected.


[]

In [66]:
df = pd.read_sql('SELECT * FROM cyclisticb;', 'sqlite:///simsimdb.db')


In [67]:
df.to_csv('cyclistic1.csv', index=False)